In [191]:
notas = [
        (100, "ALGEBRA", 9,"2019-01-13"),
        (100, "ANALISIS", 8,"2019-01-19"),
        (100, "ALGORITMOS", 9,"2018-01-28"),
        (110, "ALGORITMOS", 9,"2018-01-24"),
        (110, "ALGEBRA", 8,"2016-01-25"),
        (130, "ANALISIS", 2,"2019-01-26"),
        (120, "ALGORITMOS", 6,"2018-01-27"),
        (120, "ALGEBRA", 10,"2019-01-30"),
        (120, "ANALISIS", 9,"2018-01-31"),
        (100, "FISICA", 9,"2019-01-28"),
        (130, "ANALISIS", 3,"2018-01-28"),
        (140, "ANALISIS", 9,"2016-01-28")
]


rddnotas = sc.parallelize(notas)

In [192]:
#######################################################################
#   Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años
#######################################################################

In [193]:
alumnos_regulares = rddnotas.filter(lambda x: (x[2] >= 4) & (x[3] > "2017"))

In [194]:
alumnos_regulares = alumnos_regulares.map(lambda x: (x[0], "Aprobado"))

In [195]:
alumnos_regulares = alumnos_regulares.reduceByKey(lambda x,y: x)

In [196]:
alumnos_regulares.collect()

[(100, 'Aprobado'), (120, 'Aprobado'), (110, 'Aprobado')]

In [197]:
#######################################################################
#   Un RDD conteniendo el promedio de notas de cada alumno de la forma 
#   (padrón, promedio)
#######################################################################

In [198]:
#  Me quedo con el padron y la nota, agrego un 1 para contar

notas_por_padron = rddnotas.map(lambda x: (x[0], (x[2],1)))

In [199]:
# Sumo las notas y los 1

notas_por_padron = notas_por_padron.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))

In [200]:
# Calculo promedio como division entre acumulado de notas y los 1

notas_por_padron = notas_por_padron.map(lambda x: (x[0], x[1][0] / x[1][1]))

In [201]:
# Tomo los primeros 5 ordenados descendentemente por nota promedio

notas_por_padron.takeOrdered(5, key = lambda x: - x[1])

[(140, 9.0), (100, 8.75), (110, 8.5), (120, 8.333333333333334), (130, 2.5)]

In [202]:
#######################################################################
# El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un
# segundo RDD alumnos con registros (padron, nombre y apellido).
#######################################################################

In [203]:
nombres = [
        (100, "Lorenzo Gimenez"),
        (110, "Gorenzo Limenez"),
        (120, "Rogenzo Limezen"),
        (130, "Zolerno Ginezem"),
        (140, "Leronzo Geminez"),
]


rddnombres = sc.parallelize(nombres)

In [204]:
#  Me quedo con el padron y la nota, agrego un 1 para contar

notas_por_padron = rddnotas.map(lambda x: (x[0], (x[2],1)))
notas_por_padron.collect()

[(100, (9, 1)),
 (100, (8, 1)),
 (100, (9, 1)),
 (110, (9, 1)),
 (110, (8, 1)),
 (130, (2, 1)),
 (120, (6, 1)),
 (120, (10, 1)),
 (120, (9, 1)),
 (100, (9, 1)),
 (130, (3, 1)),
 (140, (9, 1))]

In [205]:
# Sumo las notas y los 1

notas_por_padron = notas_por_padron.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))
notas_por_padron.collect()

[(100, (35, 4)), (120, (25, 3)), (140, (9, 1)), (110, (17, 2)), (130, (5, 2))]

In [206]:
# Agrego los nombres
notas_por_nombre = notas_por_padron.join(rddnombres)

In [207]:
#notas_por_nombre = notas_por_nombre.map(lambda x:)
promedios_por_nombre = notas_por_nombre.map(lambda x: (x[1][1],x[1][0][0]/x[1][0][1]))
promedios_por_nombre.collect()

[('Rogenzo Limezen', 8.333333333333334),
 ('Zolerno Ginezem', 2.5),
 ('Lorenzo Gimenez', 8.75),
 ('Leronzo Geminez', 9.0),
 ('Gorenzo Limenez', 8.5)]

In [208]:
# Tomo el primer padron ordenado por promedio

promedios_por_nombre.takeOrdered(1, key = lambda x: - x[1])

[('Leronzo Geminez', 9.0)]